## Data preprocessing
1. Import the observations built from single-cell data.
2. Standardize the observations gene names.
3. Get interaction graph from DoRothEA database.
4. Standardize the interaction graph gene names.
5. Eliminate genes with both no influence on others into the interaction graph and no expression value into the single-cell data.

## BoNesis selection of components

### 1. Import the observations built from single-cell data:

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_nesto = pd.read_csv("nestorowa_binarizedObservations.csv", sep=",", index_col=[0])

In [3]:
df_nesto

,1110008L16Rik,1110059E24Rik,1200007C13Rik,1300017J02Rik,1500005C15Rik,1600014C10Rik,1600020E01Rik,1700006J14Rik,1700017B05Rik,1700024P16Rik,...,Zscan21,Zscan22,Zscan29,Zswim3,Zswim4,Zufsp,Zxdb,Zxdc,Zyx,Zzz3
S1,0,0,NaN,NaN,NaN,0,0,0,0,NaN,...,0,0,0,NaN,NaN,0,NaN,NaN,1,0
S4,0,1,NaN,NaN,NaN,1,0,0,0,NaN,...,0,0,0,NaN,NaN,0,NaN,NaN,1,0
S0,0,1,NaN,NaN,NaN,0,0,0,0,NaN,...,0,0,0,NaN,NaN,0,NaN,NaN,1,0
S2,0,0,NaN,NaN,NaN,1,1,0,0,NaN,...,0,0,0,NaN,NaN,0,NaN,NaN,1,0
S5,0,0,NaN,NaN,NaN,0,0,1,0,NaN,...,0,0,0,NaN,NaN,0,NaN,NaN,1,0
S3,0,0,NaN,NaN,NaN,0,0,0,1,NaN,...,0,0,0,NaN,NaN,0,NaN,NaN,1,1


In [4]:
observations_from_singlecell_nestorowa = df_nesto.to_dict(orient="index")

In [ ]:
observations_from_singlecell_nestorowa

### 2. Standardize the observations gene names.

In [7]:
import os
from typing import List, Set, Dict, Tuple


def get_dict_matching_synonyms_to_refgenename(path_NCBIgenedata: str) -> Dict:
    """
    Create a dictionary matching each possible gene name to its NCBI symbol.
    
    Particularity:
    Creation of a temporary file for speeding up the task facing a large matrix from NCBI, the parsing of the NCBI gene data is run with awk. A temporary file is then created.
    
    INPUT
        path_NCBIgenedata: path to the NCBI gene data
    OUTPUT
        dictionary (key: gene name, value: reference gene name (being the NCBI symbol))
    """
    
    # Parse the downloaded NCBI gene data:
    path_NCBIgenedata_cut = f"{path_NCBIgenedata}_cut"
    command_parsing = "awk -F'\t' '{print $3 \"\t\" $5 \"\t\" $11}' " + path_NCBIgenedata + " | tr \| '\t' > " + path_NCBIgenedata_cut + " ; sed -i 1d " + path_NCBIgenedata_cut
    os.system(command_parsing)
    
    # Extract gene data information:    
    gene_synonyms_dict = dict()
    symbols = set()

    with open (path_NCBIgenedata_cut, "r") as file_synonyms:
        for gene in file_synonyms:
            gene = gene.strip().upper()
            gene_symbols_list = gene.split("\t")
            #extract reference gene symbol:
            ncbi_symbol = gene_symbols_list.pop(0)
            #delete non-informative synonyms:
            res = [syn for syn in gene_symbols_list if (syn != "-" and syn != ncbi_symbol)]

            #create the dictionnary matching each symbol to its reference gene symbol (NCBI symbol):
            gene_synonyms_dict[ncbi_symbol] = ncbi_symbol
            symbols.add(ncbi_symbol)

            for gene in res:
                if gene not in symbols:
                    # Warning with NCBI list of synonyms:
                    # A noun can be the synonym of several symbols.
                    # Arbitrary, the choosen one is the first.
                    gene_synonyms_dict[gene] = ncbi_symbol
                    
    os.system(f"rm {path_NCBIgenedata_cut}")
    return gene_synonyms_dict


def get_reference_gene_name(gene_name: str, dict_synonyms: dict) -> str:
    """
    Given a gene name, return its reference name.
    INPUT
        dict_synonyms
        gene_name: the gene name you want its reference name
    OUTPUT
        the synonym considered as the reference name
    """
    gene_name = gene_name.upper()
    if gene_name in dict_synonyms:
        return dict_synonyms[gene_name]
    return gene_name

In [8]:
def standardize_genename_in_dict_of_observations(observations_dict: Dict, path_NCBIgenedata: str) -> Dict:
    """
    Create a copy of the input dict of observations, with each gene name replaced by its reference name (NCBI symbol).
    
    Require the following functions:
        get_dict_matching_synonyms_to_refgenename
        get_reference_gene_name
        
    INPUT
        observations_dict: dict (key = observation identifier, value = dict (key = genename, value = gene status))
        path_NCBIgenedata: path to the NCBI gene data
    OUTPUT
        dict (key = observation identifier, value = dict (key = reference genename, value = gene status))
    """
    
    # Get gene data information:
    gene_synonyms_dict = get_dict_matching_synonyms_to_refgenename(path_NCBIgenedata)
    
    # Copy the dict of observations by replacing each genename by its reference genename (NCBI symbol) into it:
    standardized_observations_dict = dict()
    
    for k,v in observations_dict.items():
        standardized_observations_dict[k] = dict()
        for component, status in v.items():
            standardized_component = get_reference_gene_name(component, gene_synonyms_dict)
            standardized_observations_dict[k][standardized_component] = status
    
    return standardized_observations_dict

In [9]:
# Copy the dict of observations by replacing each gene name by its NCBI symbol:
standardized_observations_from_singlecell_nestorowa = standardize_genename_in_dict_of_observations(observations_from_singlecell_nestorowa, "Mus_musculus.gene_info.20221005.tsv")

In [10]:
# Visualisation of the matrix with standardized gene names:
df_standardized = pd.DataFrame.from_dict(standardized_observations_from_singlecell_nestorowa, orient="index").fillna('')
df_standardized

,PRORP,1110059E24RIK,1200007C13RIK,INHCA,1500005C15RIK,1600014C10RIK,1600020E01RIK,1700006J14RIK,1700017B05RIK,FYB2,...,ZSCAN21,ZSCAN22,ZSCAN29,ZSWIM3,ZSWIM4,ZUP1,ZXDB,ZXDC,ZYX,ZZZ3
S1,0,0,,,,0,0,0,0,,...,0,0,0,,,0,,,1,0
S4,0,1,,,,1,0,0,0,,...,0,0,0,,,0,,,1,0
S0,0,1,,,,0,0,0,0,,...,0,0,0,,,0,,,1,0
S2,0,0,,,,1,1,0,0,,...,0,0,0,,,0,,,1,0
S5,0,0,,,,0,0,1,0,,...,0,0,0,,,0,,,1,0
S3,0,0,,,,0,0,0,1,,...,0,0,0,,,0,,,1,1


In [11]:
# Set of the genes in this single-cell dataset:
standardized_genenames_in_singlecell = set(df_standardized)

### 3. Get interaction graph from DoRothEA database.

In [12]:
import os
import datetime

def dorothea_extraction(organism: str="human", confidence: str="AB", directory_output: str="./"):
    ''' Store in a SIF file the subnetwork from DoRothEA database about mus musculus given the confidence on edges.
    INPUT
        organism: string that can be human or mouse
        confidence: string in the set A, AB (default), ABC, ABCD, ABCDE
        output directory: the current one by default
    OUTPUT
        SIF file in the directory_output, under the format "YYYY-MM-DD_dorotheaX.sif" with X the confidence given in argument
    '''
    
    assert organism == 'human' or organism == 'mouse', f"organism must be human or mouse"
    
    date = datetime.datetime.now()
    
    import rpy2.robjects as robjects
    import rpy2.robjects.packages as rpackages

    if confidence == "A":
        confidenceR = '"A"'
    elif confidence == "AB":
        confidenceR = '"A","B"'
    elif confidence == "ABC":
        confidenceR = '"A","B","C"'
    elif confidence == "ABCD":
        confidenceR = '"A","B","C","D"'
    elif confidence == "ABCDE":
        confidenceR = '"A","B","C","D","E"'
    else:
        raise InputError("Incorrect argument: confidence for edges can be A, AB, ABC, ABCD, ABCDE")

    #robjects.r('''
    #    if (!requireNamespace("BiocManager", quietly = TRUE))
    #        install.packages("BiocManager")
    #    BiocManager::install("dorothea")
    #''')
    
    robjects.r('''
        library(dorothea)
        subset_dth = dorothea_{0}[dorothea_{0}$confidence %in% c({1}), ]
        '''.format('hs' if organism=='human' else 'mm', confidenceR))
    robjects.r('''
        df = data.frame(source = subset_dth$tf,
                        sign = subset_dth$mor,
                        target = subset_dth$target)
        write.table(df, file="{0}dorothea{2}_{3}_{1}.sif", sep = "\t", col.names = FALSE, row.names = FALSE, quote = FALSE)
        '''.format(directory_output, date.strftime("%Y%m%d"), confidence, organism))

In [13]:
dorothea_extraction(organism="mouse", confidence="ABC")

### 4. Standardize the interaction graph gene names.

In [14]:
import os
from typing import List, Set, Dict, Tuple


def get_dict_matching_synonyms_to_refgenename(path_NCBIgenedata: str) -> Dict:
    """
    Create a dictionary matching each possible gene name to its NCBI symbol.
    
    Particularity:
    Creation of a temporary file for speeding up the task facing a large matrix from NCBI, the parsing of the NCBI gene data is run with awk. A temporary file is then created.
    
    INPUT
        path_NCBIgenedata: path to the NCBI gene data
    OUTPUT
        dictionary (key: gene name, value: reference gene name (being the NCBI symbol))
    """
    
    # Parse the downloaded NCBI gene data:
    path_NCBIgenedata_cut = f"{path_NCBIgenedata}_cut"
    command_parsing = "awk -F'\t' '{print $3 \"\t\" $5 \"\t\" $11}' " + path_NCBIgenedata + " | tr \| '\t' > " + path_NCBIgenedata_cut + " ; sed -i 1d " + path_NCBIgenedata_cut
    os.system(command_parsing)
    
    # Extract gene data information:    
    gene_synonyms_dict = dict()
    symbols = set()

    with open (path_NCBIgenedata_cut, "r") as file_synonyms:
        for gene in file_synonyms:
            gene = gene.strip().upper()
            gene_symbols_list = gene.split("\t")
            #extract reference gene symbol:
            ncbi_symbol = gene_symbols_list.pop(0)
            #delete non-informative synonyms:
            res = [syn for syn in gene_symbols_list if (syn != "-" and syn != ncbi_symbol)]

            #create the dictionnary matching each symbol to its reference gene symbol:
            gene_synonyms_dict[ncbi_symbol] = ncbi_symbol
            symbols.add(ncbi_symbol)

            for gene in res:
                if gene not in symbols:
                    # Warning with NCBI list of synonyms:
                    # A noun can be the synonym of several symbols.
                    # Arbitrary, the choosen one is the first.
                    gene_synonyms_dict[gene] = ncbi_symbol
                    
    os.system(f"rm {path_NCBIgenedata_cut}")
    return gene_synonyms_dict


def get_reference_gene_name(gene_name: str, dict_synonyms: dict) -> str:
    """
    Given a gene name, return its reference name.
    INPUT
        dict_synonyms
        gene_name: the gene name you want its reference name
    OUTPUT
        the synonym considered as the reference name
    """
    gene_name = gene_name.upper()
    if gene_name in dict_synonyms:
        return dict_synonyms[gene_name]
    return gene_name

In [15]:
def standardize_genename_in_file(path_input: str, path_NCBIgenedata: str, columns_to_standardize: List or Set[str] = [0], sep = "\t"):
    """
    Create a copy of the input file, with each gene name replaced by its reference (NCBI symbol) in the column precised in argument.
    
    Require the following functions:
        get_dict_matching_synonyms_to_refgenename
        get_reference_gene_name
       
    INPUT
        path_input: path to the input file in which the names must be standardized.
        path_NCBIgenedata: path to the NCBI gene data.
        columns_to_standardize : the columns containing gene names we want to standardize. Columns must start at index 0.
        sep: the field separator into the input SIF file (the gene data file provided by NCBI is a TSV).
    OUTPUT
        copy of the input file but with each gene into the columns_to_standardize named by its reference gene name (capitalized NCBI symbol). Named as the input file with at its end the extension "_reference-gene-names".
    """
    
    # Get gene data information:
    gene_synonyms_dict = get_dict_matching_synonyms_to_refgenename(path_NCBIgenedata)
    
    # Replace gene name with reference gene name into the columns_to_standardize of the input file:
    cols_check = set() #put all elements of columns_to_standardize in a set for complexity 
    for c in columns_to_standardize:
        cols_check.add(c)
    
    with open(path_input, "r") as inputfile:
        to_write = []
        for ligne in inputfile.read().split("\n"):
            if len(ligne) > 1:
                cols = ligne.split(sep)
                ligne_output = ""
                id_col = 0
                for col in cols[:-1]:
                    if id_col in cols_check:
                        ligne_output += get_reference_gene_name(col, gene_synonyms_dict) + sep
                    else:
                        ligne_output += col + sep
                    id_col += 1
                if id_col in cols_check:
                    ligne_output += get_reference_gene_name(cols[-1], gene_synonyms_dict)
                else:
                    ligne_output += cols[-1]
                ligne_output += "\n"
                to_write.append(ligne_output)
    
    with open(f"{path_input}_reference-gene-names", "x") as outputfile:
        for ligne in to_write:
            outputfile.write(ligne)

In [16]:
# Copy of 2022-XX-XX_dorotheaABC_mouse.sif by replacing each genename with its NCBI symbol
standardize_genename_in_file("dorotheaABC_mouse_20221129.sif", "Mus_musculus.gene_info.20221005.tsv", (0,2), "\t")

### 5. Eliminate genes with both no influence on others into the interaction graph and no expression value into the single-cell data.

In [17]:
sources = set()  #stores the sources of edges
targets = set()  #stored the targets of edges
with open("dorotheaABC_mouse_20221129.sif_reference-gene-names", "r") as f:
    for edge in f:
        edge = edge.strip().split()
        sources.add(edge[0])
        targets.add(edge[2])

In [18]:
tf_tf_interactions = sources.intersection(targets)  #Which targets are also sources?

print(f"TF 'source' which are also 'targets': {len(tf_tf_interactions)}\n\
Number of TF with expression value in the single-cell dataset: {len(sources.intersection(standardized_genenames_in_singlecell))}\n\
Number of TF 'targets' with expression value in the single-cell dataset: {len(tf_tf_interactions.intersection(standardized_genenames_in_singlecell))}")

TF 'source' which are also 'targets': 202
Number of TF with expression value in the single-cell dataset: 79
Number of TF 'targets' with expression value in the single-cell dataset: 63


In [19]:
with open("dorotheaABC_mouse_20221129.sif_reference-gene-names", "r") as f_in:
    with open("standardized_dorothea_20221129.sif", "w") as f_out:
        # In path_fichier_noms_NCBI, a line = an edge.
        # A line is written in the output file if:
        # the edge source is a target of at least one edge
        # the edge target is a source of at least one edge or belongs to the Nestorowa list of genes
        for line in f_in:
            edge = line.strip().split()
            if edge[2] in tf_tf_interactions or edge[2] in standardized_genenames_in_singlecell:
                f_out.write(line)

In [20]:
import networkx as nx

df = pd.read_csv("standardized_dorothea_20221129.sif", header=None, names=("in", "sign", "out"), sep="\t")
tf_tf_nesto = nx.from_pandas_edgelist(df, "in", "out", ["sign"], nx.MultiDiGraph())

print(f"TF→TF & TF→Nesto graph: {len(tf_tf_nesto.nodes())} nodes, {len(tf_tf_nesto.edges())} edges\n\
{len(set(tf_tf_nesto).intersection(standardized_genenames_in_singlecell))} nodes observed in Nestorowa dataset")

TF→TF & TF→Nesto graph: 1717 nodes, 4766 edges
1525 nodes observed in Nestorowa dataset


In [21]:
import os

os.system("rm dorotheaABC_mouse_20221129.sif dorotheaABC_mouse_20221129.sif_reference-gene-names")

0

## BoNesis selection of components

In [22]:
import bonesis

In [23]:
standardized_dorothea = bonesis.InfluenceGraph.from_sif("standardized_dorothea_20221129.sif", maxclause=8, allow_skipping_nodes=True, canonic=False)

In [24]:
bo = bonesis.BoNesis(standardized_dorothea, standardized_observations_from_singlecell_nestorowa)

In [25]:
print(f"domain: {len(standardized_dorothea.nodes())} nodes, {len(standardized_dorothea.edges())} edges")

domain: 1717 nodes, 4835 edges


In [26]:
standardized_observations_from_singlecell_nestorowa

{'S1': {'PRORP': 0,
  '1110059E24RIK': 0,
  '1200007C13RIK': nan,
  'INHCA': nan,
  '1500005C15RIK': nan,
  '1600014C10RIK': 0,
  '1600020E01RIK': 0,
  '1700006J14RIK': 0,
  '1700017B05RIK': 0,
  'FYB2': nan,
  '1700028E10RIK': nan,
  '1700029J07RIK': nan,
  '1700030K09RIK': 0,
  '1700065D16RIK': nan,
  '1700066M21RIK': nan,
  '1700096K18RIK': nan,
  '1810010H24RIK': nan,
  'TMEM273': nan,
  'CRACDL': nan,
  '2010315B03RIK': 0,
  '2210016F16RIK': 0,
  '2210016L21RIK': 0,
  '2310057M21RIK': 0,
  '2510009E07RIK': nan,
  '2610008E11RIK': nan,
  '2610021A01RIK': 0,
  '2610035D17RIK': 0,
  '2610044O15RIK8': 0,
  'CFAP418': 0,
  '2610307P16RIK': 0,
  '2700029L08RIK': nan,
  '2810013P06RIK': 0,
  '2810021J22RIK': nan,
  '2810029C07RIK': nan,
  '2810414N06RIK': nan,
  '2810428J06RIK': nan,
  'CEROX1': nan,
  'RESF1': 0,
  '2900005J15RIK': nan,
  '2900018N21RIK': nan,
  '2900092N22RIK': nan,
  'C9ORF72': nan,
  'ARMH4': nan,
  '4632404H12RIK': nan,
  '4632427E13RIK': 0,
  '4732440D04RIK': nan,


### Dynamics
<img src="trajectoire.png" alt="nestorowa stream trajectory" style="width:40%;"/>

#### fixpoints

In [28]:
s2 = bo.fixed(~bo.obs("S2"))
s4 = bo.fixed(~bo.obs("S4"))
s5 = bo.fixed(~bo.obs("S5"))
s2 != s4
s5 != s4
s2 != s5;

#### positive reachability

In [29]:
~bo.obs('S1') >= ~bo.obs('S0') >= s2
~bo.obs('S0') >= ~bo.obs('S3') >= s4
~bo.obs('S3') >= s5;

#### negative reachability

In [30]:
~bo.obs("S3") / s2;

### Optimization & view

In [31]:
bo.maximize_nodes()
bo.maximize_strong_constants()

<bonesis.language.ManagedIface.__init__.<locals>.managed.<locals>.Managed at 0x7fa3ce2c9a00>

In [ ]:
view = bonesis.NonStrongConstantNodesView(bo, mode="opt")

In [ ]:
view.standalone(output_filename=f"maxnodes_maxstrongconstant.sh")